# Are useful votes on a review biased?

Do users actually read a review before voting it useful? Or is their decision biased based on the cool, funny and useful votes previously received by the review?

In [2]:
import pandas as pd
import numpy as np

from sklearn import metrics
from sklearn.utils import shuffle
from sklearn.preprocessing import normalize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC

from scipy.sparse import csr_matrix, csc_matrix, hstack, vstack
SAMPLE_SIZE = 500

In [3]:
reviews = pd.read_csv('dataset/review.csv', usecols=['text', 'useful', 'cool'])

We need to make sure that we have equal number of data points for each label. We use the mean of `useful` column as our threshold, any review with a `useful` vote lesser than the mean is labeled *not useful* and anything equal or greater than the mean is labeled *useful*.

The following cell randomly samples for a predefined number of *useful* and *not useful* reviews which are then concatenated together and the resulting df is then shuffled.

In [4]:
THRESHOLD = round(reviews['useful'].mean()) 

useful_reviews = reviews.loc[reviews['useful'] >= THRESHOLD].sample(SAMPLE_SIZE)
not_useful_reviews = reviews.loc[reviews['useful'] < THRESHOLD].sample(SAMPLE_SIZE)
print("useful_reviews shape: ", useful_reviews.shape)
print("not_useful_reviews shape: ", not_useful_reviews.shape)

reviews = shuffle(pd.concat([useful_reviews, not_useful_reviews]))
reviews.head()

useful_reviews shape:  (500, 3)
not_useful_reviews shape:  (500, 3)


,text,useful,cool
218396,The hamburger was fresh and so were the toppin...,0,0
5218305,If you're looking for help in the Home departm...,0,0
905548,Heard about this place from friends how great ...,0,0
4914250,"Sushi in big wooden boats!\n\nI have to say, I...",4,1
4019227,We had a great server named Herschel who defin...,0,1


# Preprocessing

For `text` a custom `analyzer` method is written which:
1. remove all punctuations and
3. removes new line characters (escape sequence)

The feature extraction (`CountVectorize` and `TfidfVectorizer`) class is then set to:
2. remove accents
2. remove all stopwords
3. lowercase all words

For `useful` votes:
2. remove df row if `useful` is `NaN`

In [5]:
reviews.dropna(inplace=True)
print("reviews shape after dropping NaN values: ", reviews.shape)

reviews shape after dropping NaN values:  (1000, 3)


In [6]:
import string
import re

RE_NEWLINE = '\n+'
PUNCTUATIONS = string.punctuation

def review_process(review):
    no_newline = re.sub(RE_NEWLINE, '', review)
    no_punc = ''.join([char for char in no_newline if char not in PUNCTUATIONS])
    
    return no_punc

In [7]:
test = "a str.. with! some @2 punctuations 6546721 and numbers \n [] \n"
print(review_process(test))

a str with some 2 punctuations 6546721 and numbers   


In [8]:
vect = TfidfVectorizer(strip_accents='ascii', preprocessor=review_process, stop_words='english' )
vect

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2',
        preprocessor=<function review_process at 0x14b1c5bf8>,
        smooth_idf=True, stop_words='english', strip_accents='ascii',
        sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, use_idf=True, vocabulary=None)

# Generating labels

In [11]:
import math

NOT_USEFUL = 0
USEFUL = 1

def labeler(vote):
    if math.floor(vote) < THRESHOLD:
        return NOT_USEFUL
    else:
        return USEFUL

In [12]:
reviews['label'] = reviews['useful'].apply(labeler)
    
reviews.head()

,text,useful,cool,label
218396,The hamburger was fresh and so were the toppin...,0,0,0
5218305,If you're looking for help in the Home departm...,0,0,0
905548,Heard about this place from friends how great ...,0,0,0
4914250,"Sushi in big wooden boats!\n\nI have to say, I...",4,1,1
4019227,We had a great server named Herschel who defin...,0,1,0


# Generating test train splits

In [13]:
X_train, X_test, Y_train, Y_test = train_test_split(reviews[['text', 'useful', 'cool']], reviews['label'], test_size=0.4)

# Instantiate classifiers

We will be using 3 classifiers for this analysis namely, *Multinomial Naive Bayes*, *Linear SVM* and *Random Forest*.

In [14]:
mn_clas = MultinomialNB()
print(mn_clas)
print("\n")

svc_clas = LinearSVC()
print(svc_clas)
print("\n")

rf_clas = RandomForestClassifier()
print(rf_clas)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)


LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)


# Feature selection

As we increase the number of data points, the vocabulary of our classifiers will increase which means the number of features extracted will also increase. For a large numebr of data points, this will be memory intensive and will also result in longer training time.

To improve the performance of our program, we will remove all zero-variance features.

In [19]:
sel = VarianceThreshold()
sel

VarianceThreshold(threshold=0.0)

# Model 1: training with only text

In [28]:
# feature extraction
X1 = vect.fit_transform(X_train['text'])
X1_test = vect.transform(X_test['text'])
print("shape of X1 after feature extraction: ", X1.shape)

# feature selection
X1 = sel.fit_transform(X1)
X1_test = sel.transform(X1_test)
print("shape of X1_test after feature extraction: ", X1.shape)

shape of X1 after feature extraction:  (600, 8663)
shape of X1_test after feature extraction:  (600, 8663)


In [29]:
type(X1)

scipy.sparse.csr.csr_matrix

In [30]:
mn_clas.fit(X1, Y_train)
svc_clas.fit(X1, Y_train)
rf_clas.fit(X1, Y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [31]:
print("Naive Bayes score: ", mn_clas.score(X1_test, Y_test))
print("Linear SVC score: ", svc_clas.score(X1_test, Y_test))
print("Random Forest score: ", rf_clas.score(X1_test, Y_test))

Naive Bayes score:  0.5675
Linear SVC score:  0.555
Random Forest score:  0.5575


# Model 2: training with text and useful votes

Since X now contains mixed dtypes, we need to be a bit clever. First, we convert `useful` column for train and test into sparse matrix and *I2* normalize them (I2 is the default for `TfidfVectorizer` so that's what we use here as well).

The rest is straight forward, we create our updated X by stacking the `useful` sparse matrices with X horizontally (such that the number of feature increases). 

In [146]:
useful_sparse_train = normalize(csr_matrix(X_train['useful']))
useful_sparse_test = normalize(csr_matrix(X_test['useful']))
print("type of useful_sparse_train: ", type(useful_sparse_train))
print("shape of useful_sparse_train: ", useful_sparse_train.shape)

type of useful_sparse_train:  <class 'scipy.sparse.csr.csr_matrix'>
shape of useful_sparse_train:  (1, 600)


In [158]:
# training
X2 = hstack([X1, useful_sparse_train.T])
print("shape of X2: ", X2.shape)

# testing
X2_test = hstack([X1_test, useful_sparse_test.T])

shape of X2:  (600, 8666)


In [148]:
mn_clas.fit(X2, Y_train)
svc_clas.fit(X2, Y_train)
rf_clas.fit(X2, Y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [149]:
print("Naive Bayes score: ", mn_clas.score(X2_test, Y_test))
print("Linear SVC score: ", svc_clas.score(X2_test, Y_test))
print("Random Forest score: ", rf_clas.score(X2_test, Y_test))

Naive Bayes score:  0.5675
Linear SVC score:  0.66
Random Forest score:  0.9625


# Model 3: training with text and cool votes

Next we will carry out the analysis with text and `cool` votes. We choose `cool` votes since it had the second highest correlation with `useful` (see exploration.ipynb).

In [152]:
cool_sparse_train = normalize(csr_matrix(X_train['cool']))
cool_sparse_test = normalize(csr_matrix(X_test['cool']))

In [159]:
# training
X3 = hstack([X1, cool_sparse_train.T])
print("shape of X3: ", X3.shape)

# testing
X3_test = hstack([X1_test, cool_sparse_test.T])

shape of X3:  (600, 8666)


In [154]:
mn_clas.fit(X3, Y_train)
svc_clas.fit(X3, Y_train)
rf_clas.fit(X3, Y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [155]:
print("Naive Bayes score: ", mn_clas.score(X3_test, Y_test))
print("Linear SVC score: ", svc_clas.score(X3_test, Y_test))
print("Random Forest score: ", rf_clas.score(X3_test, Y_test))

Naive Bayes score:  0.4875
Linear SVC score:  0.5175
Random Forest score:  0.6725


# Model 4: training with text, useful and cool votes

Finally we will carry out the analysis with text, `useful` & `cool` votes.

In [157]:
uc_sparse_train = normalize(csr_matrix(X_train[['cool', 'useful']]))
uc_sparse_test = normalize(csr_matrix(X_test[['cool', 'useful']]))
print("shape of uc_sparse_train: ", uc_sparse_train.shape)

shape of uc_sparse_train:  (600, 2)


In [160]:
# training
X4 = hstack([X1, uc_sparse_train])
print("shape of X4: ", X4.shape)

# testing
X4_test = hstack([X1_test, uc_sparse_test])

shape of X4:  (600, 8667)


In [161]:
mn_clas.fit(X4, Y_train)
svc_clas.fit(X4, Y_train)
rf_clas.fit(X4, Y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [162]:
print("Naive Bayes score: ", mn_clas.score(X4_test, Y_test))
print("Linear SVC score: ", svc_clas.score(X4_test, Y_test))
print("Random Forest score: ", rf_clas.score(X4_test, Y_test))

Naive Bayes score:  0.92
Linear SVC score:  0.9975
Random Forest score:  0.865
